In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime
from functools import partial
import os

In [ ]:
# 초기 변수 및 레이아웃
todo_items = []
layout_input = widgets.Layout(width='100%')
layout_label = widgets.Layout(width='80px')
layout_button = widgets.Layout(width='100px', height='32px')
layout_button_wide = widgets.Layout(width='220px', height='32px')
layout_progress_bar = widgets.Layout(width='40%', height='24px')
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=layout_progress_bar)
overall_progress_bar.style.bar_color = 'black'
overall_progress_label = widgets.Label(value='(0%)')
todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

In [ ]:
def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

In [ ]:
def refresh_list():
    todo_list_box.children = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_input)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_input)
        progress_bar.style.bar_color = 'blue'
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_input)
        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_input)
        priority_label = widgets.Label(value=item['priority'], layout=layout_input)
        complete_btn = widgets.Button(description='완료✅', layout=widgets.Layout(width='70px'))
        delete_btn = widgets.Button(description='삭제❌', layout=widgets.Layout(width='70px'))

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        progress_slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        row = widgets.VBox([
            widgets.HBox([widgets.Label('할 일', layout=layout_label), title]),
            widgets.HBox([widgets.Label('진행률', layout=layout_label), progress_bar]),
            widgets.HBox([widgets.Label('', layout=layout_label), progress_slider]),
            widgets.HBox([widgets.Label('입력 마감일', layout=layout_label), due_label]),
            widgets.HBox([widgets.Label('중요도', layout=layout_label), priority_label]),
            widgets.HBox([widgets.Label('작업', layout=layout_label), widgets.HBox([complete_btn, delete_btn])])
        ], layout=widgets.Layout(margin='10px 0'))

        todo_list_box.children += (row,)

In [ ]:
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_input)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_input)
due_input = widgets.DatePicker(layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_input)
add_button = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='440px'))

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

In [ ]:
def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {**i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None} for i in todo_items
        ], f)

def load_data(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                update_overall_progress()
                refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_button)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_button)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

In [ ]:
header_label = widgets.Label(value='DAVID mk19 - 즉시 실행 가능')
input_row = widgets.VBox([
    widgets.HBox([widgets.Label('할 일', layout=layout_label), title_input]),
    widgets.HBox([widgets.Label('진행률', layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label('마감일', layout=layout_label), due_input]),
    widgets.HBox([widgets.Label('중요도', layout=layout_label), priority_input]),
    widgets.HBox([widgets.Label('', layout=layout_label), add_button]),
])

buttons_row = widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center'))
progress_row = widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center'))

display(widgets.VBox([
    header_label,
    input_row,
    buttons_row,
    progress_row,
    todo_list_box
]))
update_overall_progress()
refresh_list()